In [3]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

In [4]:
from modules.data.conll2003.prc import conll2003_preprocess

In [7]:
data_dir = "../modules/data/conll2003/"

In [8]:
conll2003_preprocess(data_dir)

## IO markup

### Train

In [10]:
from modules.data import bert_data

I1216 17:30:17.692191  8116 file_utils.py:39] PyTorch version 1.2.0 available.
I1216 17:30:19.266733  8116 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [12]:
import os
data = bert_data.LearnData.create(
    train_df_path=os.path.join(data_dir,"eng.train.train.csv"),
    valid_df_path=os.path.join(data_dir,"eng.testa.dev.csv"),
    idx2labels_path=os.path.join(data_dir,"idx2labels.txt"),
    clear_cache=True
)

I1216 17:32:55.252265  8116 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt not found in cache or force_download set to True, downloading to C:\Users\Chongjiu\AppData\Local\Temp\tmp0agp95h5
100%|███████████████████████████████████████████████████████████████████████| 995526/995526 [00:02<00:00, 487623.91B/s]
I1216 17:32:58.492809  8116 file_utils.py:309] copying C:\Users\Chongjiu\AppData\Local\Temp\tmp0agp95h5 to cache at C:\Users\Chongjiu\.cache\torch\transformers\96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
I1216 17:32:58.497795  8116 file_utils.py:313] creating metadata file for C:\Users\Chongjiu\.cache\torch\transformers\96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
I1216 17:32:58.499814  8116 file_utils.py:322] removing temp file C:\Users\Chongjiu\AppDat

In [14]:
from modules.models.bert_models import BERTBiLSTMCRF

In [15]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label),
    model_name='./bert_data'
    lstm_dropout=0., crf_dropout=0.3)

I1216 17:34:13.360921  8116 file_utils.py:296] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json not found in cache or force_download set to True, downloading to C:\Users\Chongjiu\AppData\Local\Temp\tmpcs7siohh
100%|█████████████████████████████████████████████████████████████████████████████| 521/521 [00:00<00:00, 129288.39B/s]
I1216 17:34:14.612527  8116 file_utils.py:309] copying C:\Users\Chongjiu\AppData\Local\Temp\tmpcs7siohh to cache at C:\Users\Chongjiu\.cache\torch\transformers\45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200f30eef109d0a047c484
I1216 17:34:14.613551  8116 file_utils.py:313] creating metadata file for C:\Users\Chongjiu\.cache\torch\transformers\45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200f30eef109d0a047c484
I1216 17:34:14.615517  8116 file_utils.py:322] removing temp file C:\Users\Chongjiu\AppD

In [17]:
from modules.train.train import NerLearner

In [18]:
num_epochs = 100

In [19]:
learner = NerLearner(
    model, data, "../modules/models/conll2003-BERTBiLSTMCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [20]:
model.get_n_trainable_params()

2235023

In [22]:
learner.fit(epochs=num_epochs)

I1216 17:54:30.972513  8116 train.py:162] Resuming train... Current epoch 1.


RuntimeError: input must have 2 dimensions, got 1

### Predict

In [12]:
from modules.data.bert_data import get_data_loader_for_predict

In [13]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [14]:
preds = learner.predict(dl)

In [15]:
from sklearn_crfsuite.metrics import flat_classification_report

In [16]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [17]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [18]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [20]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9514    0.9509    0.9511      2016
         I_O     0.9968    0.9970    0.9969     41702
      I_MISC     0.9353    0.8974    0.9160      1257
       I_PER     0.9849    0.9825    0.9837      2856
       I_LOC     0.9697    0.9637    0.9667      1926

   micro avg     0.9917    0.9905    0.9911     49757
   macro avg     0.9676    0.9583    0.9629     49757
weighted avg     0.9916    0.9905    0.9910     49757



### Test

In [12]:
from modules.data.bert_data import get_data_loader_for_predict

In [24]:
dl = get_data_loader_for_predict(data, df_path="/home/eartemov/ae/work/conll2003/eng.testb.dev.csv")

In [25]:
preds = learner.predict(dl)

In [26]:
from sklearn_crfsuite.metrics import flat_classification_report

In [27]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [28]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [29]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [30]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8988    0.9147    0.9067      2368
         I_O     0.9952    0.9917    0.9934     37573
      I_MISC     0.8163    0.8055    0.8108       910
       I_PER     0.9759    0.9770    0.9765      2698
       I_LOC     0.9170    0.9296    0.9233      1819

   micro avg     0.9822    0.9806    0.9814     45368
   macro avg     0.9206    0.9237    0.9221     45368
weighted avg     0.9823    0.9806    0.9814     45368



In [11]:
os.path.join(data_dir,"eng.train.train.csv"),

NameError: name 'os' is not defined